In [ ]:
import os
import sys
sys.path.append('/media/mangaldeep/HDD2/workspace/MotionControl_MasterThesis')

In [ ]:
import mne
import import_ipynb

# Local Imports
from  preproc.preproc_pipeline import ica2,ssp,car,spectral_filt
from extraction.physionet_MI import extract

from data import brain_atlas  as bm

### Preprocessing

#### Remove Line noise and apply band pass

In [ ]:
#2, Spectral Filtering
rawfltrd = spectral_filt(raw, elec_lines, L_cutoff, H_cutoff)

In [ ]:
## Analysis after Spectral filter
# rawfltrd.plot(scalings=scale, clipping='transparent'); #, order=pick_ch_idx
# rawfltrd.plot(scalings='auto');
# rawfltrd.plot_psd_topo(show = True);
# rawfltrd.plot_sensors(show_names=True,kind = '3d', sphere=(0.0, 0.015, 0.033, 0.1));

In [ ]:
#3.2, Artifact Removal: CAR
rawfltrd_car = car(rawfltrd)

In [ ]:
#3.1, Artifact Removal: SSP
rawfltrd_ssp = ssp(rawfltrd_car)

In [ ]:
rawfltrd_car.plot_psd_topo();
rawfltrd_ssp.plot_psd_topo();

In [ ]:
## Analysis after Projections
# rawfltrd_ssp.plot_projs_topomap();
rawfltrd_ssp.info['projs']
# Compare after and before projection
# rawfltrd.plot(scalings=scale, clipping='transparent', order=pick_ch_idx);
# rawfltrd.plot(scalings=scale, clipping='transparent', butterfly= False, title='Raw Filtered (Without Projection))', proj= False, order=pick_ch_idx);
# rawfltrd.plot(scalings=scale, clipping='transparent', butterfly= False, title='Raw Filtered(With Projection)', proj= True, order=pick_ch_idx);

In [ ]:
#3.3, Artifact Removal: ICA
ica, rawfltrd_ica = ica2(rawfltrd)

In [ ]:
ica.exclude

In [ ]:
## Analysis after ICA
# ica.plot_components();
# ica.plot_sources(rawfltrd_ica);
# ica.plot_overlay(rawfltrd_ica, exclude = ica.exclude);
ica.plot_properties(rawfltrd_ica);
# ica.plot_scores(ex_scores);

In [ ]:
 # ica.exclude = [0,1,2,3] # manually exclude ICA components based on observation on plots above
rawfltrd_ica = ica.apply(rawfltrd_ica) # Reconstructed sensor data (In Senso space)
rawfltrd_ica.plot(scalings=scale, clipping='transparent', title='ICA  on RAW', proj= False); # , order=pick_ch_idx
# rawfltrd.plot(scalings=scale, clipping='transparent', title='Raw Filtered- w Projection', proj=True); #, order=pick_ch_idx
rawfltrd.plot(scalings=scale, clipping='transparent', title='Raw Filtered- wo Projection', proj=False);# , order=pick_ch_idx

In [ ]:
## Compare ICA vs SSP
# Comparison using data
rawfltrd_df = rawfltrd.to_data_frame(index=['time'])
rawfltrd_proj_df = rawfltrd_proj.to_data_frame(index=['time'])
rawfltrd_ica_df = rawfltrd_ica.to_data_frame(index=['time'])
rawfltrd_ica_df.compare(rawfltrd_proj_df, align_axis=0)

In [ ]:
# comparison using plots
dummy = rawfltrd.copy()
data1 = rawfltrd.get_data()
data2 = rawfltrd_ica.get_data()
data3 = rawfltrd_proj.get_data()
dummy._data = data3 - data2
dummy.plot(clipping='transparent', title='DIfference between ICA and SSP', proj=False);

### Create Epcohs from events

In [ ]:
## TODO: Choose between rawfltrd/ rawfltrd_proj/ rawfltrd_ica
# Capture events from annotations
event_data = mne.events_from_annotations(rawfltrd_proj)
event_marker, event_ids = event_data
# event_ids = dict({'base':0, 'left':1, 'right':2}) # Replacing the existing event ids

In [ ]:
# epochs1 = mne.Epochs(rawfltrd, events= event_marker, event_id= event_ids, baseline = (0,0))
epochs = mne.Epochs(raw, events= event_marker, event_id= event_ids, verbose= False, proj= True, reject = None) # Baseline is default (None,0)
# epochs.equalize_event_counts() # Shape = epochs x chan x timepnts

# Evoked data
T0 = epochs['T0'].average() # Shape = chan x timepnts
T1 = epochs['T1'].average()
T2 = epochs['T2'].average()

In [ ]:
epochs['T2'].get_data().shape

In [ ]:
(list(epochs['T0'].iter_evoked()))

In [ ]:
## Task  Analysis
task = T1
title = str(task)[11:11+2]
# task.plot_topomap();
# task.plot_white(); # Noise cov required
# task.plot_field(); # requires  surf maps
# task.plot_sensors();
task.plot_topo();
task.plot_joint(times=[0.0, 0.2, 0.3],picks=['C4','C2','C6','C1','C3','C5']);
task.plot_image(titles=f'{title} Image');
# task.plot(proj= True, titles = '{task} - Projs - True',spatial_colors=True);
# task.plot(proj= False, titles = '{task} -  Projs - False',spatial_colors=True);
# task.plot(proj= 'reconstruct', titles = '{task} -  Projs - reconstruct',spatial_colors=True);
# task.plot_topomap();
# task.plot(gfp= "only"); # population standard deviation of the signal across channels
## Compare regions
# mne.channels.combine_channels({task}, roi_dict, method='mean')
## Compare conditions
# evoked = dict(T1 = list(epochs[title].iter_evoked()), T2 = list(epochs['T2'].iter_evoked()), T0=list(epochs['T0'].iter_evoked()))
# mne.viz.plot_compare_evokeds(evoked, combine='mean');
# task_t0 = mne.combine_evoked([task, T0], weights=[1,-1])
# task_t0.plot_joint();

In [ ]:
rawfltrd_ica.ch_names[5]

In [ ]:
# epochs.to_data_frame(index=['condition', 'epoch'],long_format=True)
epochs.plot();
# epochs.plot_drop_log();

In [ ]:
# Amplitudes and latency measures
channel, latency, amplitude = T1.get_peak(mode='pos', return_amplitude=True)
print(channel, latency, amplitude)

In [ ]:
## Data whitening
noise_cov = mne.compute_covariance(epochs, tmax=0., method='shrunk', rank=None,
                                   verbose='error')
T1.plot_white(noise_cov=noise_cov); # step of scaling the whitened plots to show how well the assumption of Gaussian noise is satisfied by the data
 